# Load a fasta file containing protein sequences


In [1]:
import numpy as np
import pandas as pd

from my_library import read_fasta

# example dataset from the paper
fasta_file = 'datasets/phosphatase/phosphatase.fa'

# get the headers and sequences from the fasta file
headers = []
sequences = []

for h, s in read_fasta(fasta_file):
    headers.append(h)
    sequences.append(s)

# show the headers and sequences in the context of a dataframe
pd.DataFrame({'headers': headers, 'sequences': sequences})

,headers,sequences
0,P09923 AP_AP_AP_ALPI,PAEEENPAFWNRQAAEALDAAKKLQPIQKVAKNLILFLGDGLGVPT...
1,P05186 AP_AP_AP_ALPL,PEKEKDPKYWRDQAQETLKYALELQKLNTNVAKNVIMFLGDGMGVS...
2,P05187 AP_AP_AP_ALPP,IPVEEENPDFWNREAAEALGAAKKLQPAQTAAKNLIIFLGDGMGVS...
3,P10696 AP_AP_AP_ALPPL2,IPVEEENPDFWNRQAAEALGAAKKLQPAQTAAKNLIIFLGDGMGVS...
4,A0A2R8YDJ8|1 CC1_DSP_CDC14_CDC14A,SDRLYFATLRNRPKSTVNTHYFSIDEELVYENFYADFGPLNLAMVY...
...,...,...
199,P53041 PPPL_PPP_PPP5C_PPP5C,IESMTIEDEYSGPKLEDGKVTISFMKELMQWYKDQKKLHRKCAYQI...
200,O00743 PPPL_PPP_PPP6C_PPP6C,KRLCDYVCDLLLEESNVQPVSTPVTVCGDIHGQFYDLCELFRTGGQ...
201,O14829 PPPL_PPP_PPP7C_PPEF1,WDYVDSIDVPDSYNGPRLQFPLTCTDIDLLLEAFKEQQILHAHYVL...
202,O14830 PPPL_PPP_PPP7C_PPEF2,KKCSDYESIEVPDSYTGPRLSFPLLPDHATALVEAFRLKQQLHARY...


# Generate fixed-size embeddings for each protein sequence

In [2]:
import torch
import esm 

# specify the device for running the model
# if you have a good enough gpu, you can try "cuda" or "cuda:0"
device = 'cuda' 

# load the esm-1b protein language model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

model.eval() # disable dropout for deterministic results
model = model.to(device)

In [3]:
import sys

embeddings = []

with torch.no_grad():
    for n, s in enumerate(sequences):
        sys.stderr.write(f'Progress : {n+1} / {len(sequences)}\r')
        
        batch_labels, batch_strs, batch_tokens = batch_converter([[None, s]])
        batch_tokens = batch_tokens.to(device)
        
        # generate the full size embedding vector
        # this can be kinda big, so we won't save it for this example
        result = model(batch_tokens, repr_layers=[33], return_contacts=False)
        full_size = result["representations"][33].to('cpu')[0]
        
        # derive a fixed size embedding vector
        # there are many ways to do this, but for this example we take the mean of all residue tokens
        fixed_size = full_size[1:-1].mean(0).numpy()
        
        # save the fixed size embedding to a list
        embeddings.append(fixed_size)

# format the embeddings as a single numpy array
embeddings = np.array(embeddings)
embeddings

array([[-0.08895916,  0.1924256 ,  0.10114792, ..., -0.08377813,
        -0.04200561,  0.08479097],
       [-0.08069417,  0.15126628,  0.05401251, ..., -0.11240263,
        -0.04240455,  0.10154943],
       [-0.05566766,  0.16338356,  0.09052663, ..., -0.08684558,
        -0.04097868,  0.09436841],
       ...,
       [-0.06482267,  0.21759085,  0.01121997, ..., -0.07138941,
        -0.20048659,  0.01506873],
       [-0.10866439,  0.20918502,  0.03985881, ..., -0.09658115,
        -0.20852332, -0.0480808 ],
       [-0.09754238,  0.12654153,  0.06705502, ...,  0.02899404,
        -0.26755077, -0.20721059]], dtype=float32)

In [4]:
# show the headers, sequences, and embeddings in the context of a dataframe
pd.DataFrame({'headers': headers, 'sequences': sequences, 'embeddings': embeddings.tolist()})

,headers,sequences,embeddings
0,P09923 AP_AP_AP_ALPI,PAEEENPAFWNRQAAEALDAAKKLQPIQKVAKNLILFLGDGLGVPT...,"[-0.08895915746688843, 0.19242559373378754, 0...."
1,P05186 AP_AP_AP_ALPL,PEKEKDPKYWRDQAQETLKYALELQKLNTNVAKNVIMFLGDGMGVS...,"[-0.08069416880607605, 0.15126627683639526, 0...."
2,P05187 AP_AP_AP_ALPP,IPVEEENPDFWNREAAEALGAAKKLQPAQTAAKNLIIFLGDGMGVS...,"[-0.055667661130428314, 0.16338355839252472, 0..."
3,P10696 AP_AP_AP_ALPPL2,IPVEEENPDFWNRQAAEALGAAKKLQPAQTAAKNLIIFLGDGMGVS...,"[-0.05774325132369995, 0.17116160690784454, 0...."
4,A0A2R8YDJ8|1 CC1_DSP_CDC14_CDC14A,SDRLYFATLRNRPKSTVNTHYFSIDEELVYENFYADFGPLNLAMVY...,"[0.08296213299036026, 0.3692079484462738, 0.05..."
...,...,...,...
199,P53041 PPPL_PPP_PPP5C_PPP5C,IESMTIEDEYSGPKLEDGKVTISFMKELMQWYKDQKKLHRKCAYQI...,"[-0.1499679982662201, 0.207498237490654, 0.065..."
200,O00743 PPPL_PPP_PPP6C_PPP6C,KRLCDYVCDLLLEESNVQPVSTPVTVCGDIHGQFYDLCELFRTGGQ...,"[-0.08374017477035522, 0.14190754294395447, -0..."
201,O14829 PPPL_PPP_PPP7C_PPEF1,WDYVDSIDVPDSYNGPRLQFPLTCTDIDLLLEAFKEQQILHAHYVL...,"[-0.06482267379760742, 0.21759085357189178, 0...."
202,O14830 PPPL_PPP_PPP7C_PPEF2,KKCSDYESIEVPDSYTGPRLSFPLLPDHATALVEAFRLKQQLHARY...,"[-0.1086643859744072, 0.20918501913547516, 0.0..."


# Calculate the all-vs-all distance matrix

In [5]:
from scipy.spatial.distance import cdist

# calculate a distance matrix for the embeddings
# for this example, we will use cosine distance
distmat_embeddings = cdist(embeddings, embeddings, metric='cosine')

# Calculate an NJ Tree

In [6]:
from my_library import neighbor_joining

# neighbor joining algorithm
nj_newick_tree = neighbor_joining(distmat_embeddings, headers)

In [7]:
from ete3 import Tree, TextFace, TreeStyle, NodeStyle

# plot the tree using ete3
t = Tree(nj_newick_tree) # initialize from newick string
t.ladderize() # sort the tree

ts = TreeStyle() # set up the visual style
ts.mode = "c"
t.show(tree_style=ts)